In [24]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [25]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [26]:
x_train = x_train.reshape(-1, 28*28).astype(np.float32) / 255.0
x_test = x_test.reshape(-1, 28*28).astype(np.float32) / 255.0

In [27]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(name='w', shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='b', shape=(self.units,), initializer='zero', trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    

class MyReLu(layers.Layer):
    def __init__(self):
        super(MyReLu, self).__init__()
        
    def call(self, x):
        return tf.math.maximum(x, 0)
        
    
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)
        
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLu()
        
    def call(self, input_tensor):
        # x = tf.nn.relu(self.dense1(input_tensor))
        x = self.relu(self.dense1(input_tensor))
        return self.dense2(x)

In [28]:
# # sequential api
# model = keras.models.Sequential([
#     layers.Input(shape=(28*28)),
#     layers.Dense(512, activation='relu'),
#     layers.Dense(256, activation='relu'),
#     layers.Dense(10)
# ])

model = MyModel()
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])



In [29]:

# # functional api
# inputs = layers.Input(shape=(28*28))
# x = layers.Dense(512, activation='relu')(inputs)
# x = layers.Dense(256, activation='relu')(x)
# outputs = layers.Dense(10, activation='softmax')(x)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
#               optimizer=keras.optimizers.Adam(learning_rate=0.001),
#               metrics=['accuracy'])


# model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
# model.evaluate(x_test, y_test, batch_size=32, verbose=2)

In [34]:
# pre-trained model
model = keras.applications.InceptionV3(include_top=True)
# model.summary()
base_input = model.layers[0].input
base_output = model.layers[-1].output
final_output = layers.Dense(5)(base_output)
new_model = keras.Model(inputs=base_input, outputs=final_output)
new_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
